**Data Preprocessing**

Build Dataset

In [ ]:
import os 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import json

root_dir = '../dataset/Hindustani'
data = {"Pitch_TimeSeries":[], "CTonic":[]}

for concert in os.listdir(root_dir):
    concert_path = os.path.join(root_dir, concert)

    if os.path.isdir(concert_path): 
        for raga in os.listdir(concert_path): 
            raga_path = os.path.join(concert_path, raga)
            if os.path.isdir(raga_path): 
                pitch_file = os.path.join(raga_path, f"{raga}.pitch.txt")
                ctonic_file = os.path.join(raga_path, f"{raga}.ctonic.txt")

                with open(ctonic_file, 'r') as ctf: 
                    with open(pitch_file, 'r') as pf: 
                        pitch = pf.readlines()
                        seg_length = 3000
                        ctonic = ctf.read().strip() 

                        segments = [] 
                        curr_seg = [] 
                        ctonic_val = []

                        for line in pitch: 
                            timestamp, pitch = map(float, line.split())
                            curr_seg.append(pitch)
                            if len(curr_seg) == seg_length: 
                                segments.append(curr_seg)
                                ctonic_val.append(ctonic)
                                curr_seg = []

                        if curr_seg: 
                            segments.append(curr_seg)
                            ctonic_val.append(ctonic)

                data['Pitch_TimeSeries'].extend(segments) 
                data["CTonic"].extend(ctonic_val)    


with open('data.json', "w") as fp: 
    json.dump(data, fp)

Train RNN-LSTM

In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

import json
with open("data.json", "r") as fp:
        data = json.load(fp)

print(len(data['Pitch_TimeSeries']))
print(len(data['CTonic']))
X = np.array(data['Pitch_TimeSeries'])
y = np.array(data['CTonic'])


2024-04-10 22:53:04.647443: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


11817
11817


/var/folders/3x/zj6fq_hs7575trs1w5lb3syr0000gn/T/ipykernel_43768/2593098369.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(data['Pitch_TimeSeries'])


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [3]:
print(len(X_train[0]), len(X_test[0]), len(y_train[0]), len(y_test[0]))
print(X_train.shape)

3000 3000 10 10
(8862,)


Build Model 

In [4]:
model = keras.Sequential()
model.add(keras.layers.LSTM(4, input_shape=(3000, 1))) # hardcoded for nowww
model.add(keras.layers.Dense(64, activation='relu')) #output layer for regression
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(100, activation='softmax'))
model.compile(optimizer='adam',
                  loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 96        
                                                                 
 dense (Dense)               (None, 64)                320       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 100)               6500      
                                                                 
Total params: 6,916
Trainable params: 6,916
Non-trainable params: 0
_________________________________________________________________


2024-04-10 22:53:26.960521: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-10 22:53:26.962348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-10 22:53:26.964073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
# model = keras.Sequential()
# model.add(keras.layers.LSTM(64, input_shape=(3000, 1)))
# model.add(keras.layers.LSTM(64))
# model.add(keras.layers.Dense(64, activation='relu')) #output layer for regression
# model.add(keras.layers.Dropout(0.3))
# model.add(keras.layers.Dense(1000, activation='softmax'))
# optimiser = keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimiser,
#                   loss='mean_squared_error')
# model.summary()

Train Model

In [5]:
X_train  = keras.utils.pad_sequences(X_train, maxlen=3000, dtype='float32')
X_test = keras.utils.pad_sequences(X_test, maxlen=3000, dtype='float32')
ytofloat = np.vectorize(lambda x: float(x))
y_train = ytofloat(y_train)
y_test = ytofloat(y_test)

<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [6]:
history = model.fit(X_train, y_train, batch_size=5, epochs=3)

Epoch 1/3


2024-04-10 22:53:29.641679: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-10 22:53:29.643661: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-10 22:53:29.645883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1773/1773 [==============================] - 1367s 770ms/step - loss: 23632.0762
Epoch 2/3
1773/1773 [==============================] - 1357s 765ms/step - loss: 23632.0801
Epoch 3/3
1773/1773 [==============================] - 1366s 770ms/step - loss: 23632.1016


Evaluation


In [8]:
test_loss = model.evaluate(X_test, y_test, verbose=2)
print("Test loss:", test_loss)

2024-04-11 00:03:31.323501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 00:03:31.326064: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 00:03:31.328346: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

93/93 - 15s - loss: 23600.1055 - 15s/epoch - 162ms/step
Test loss: 23600.10546875
